In [1]:
from functions import Standings
import pandas as pd
import numpy as np
import time

### Create csv

In [2]:
# Create a dataframe with all schedules from 1979-80 to 2022-23
years = np.arange(1979, 2024)
dfs = []
for year in years :

    print(f'Fetching standings for season {year-1}-{year} ...', end = '\r')
    dfs.append(Standings(year))
    time.sleep(3)

data = pd.concat(dfs).reset_index(drop = True)
data.to_csv('data/Schedules_1979-2023.csv', index = None)

,Team,Tm,Year,W,L,W/L%,PS/G,PA/G,SRS
0,Washington Bullets,WSB,1979,54,28,0.659,114.9,109.9,4.75
1,Seattle SuperSonics,SEA,1979,52,30,0.634,106.6,103.9,2.69
2,Phoenix Suns,PHO,1979,50,32,0.610,115.4,111.7,3.55
3,San Antonio Spurs,SAS,1979,48,34,0.585,119.3,114.1,4.97
4,Kansas City Kings,KCK,1979,48,34,0.585,113.1,110.2,2.73
...,...,...,...,...,...,...,...,...,...
1241,Portland Trail Blazers,POR,2023,33,49,0.402,113.4,117.4,-3.96
1242,Charlotte Hornets,CHO,2023,27,55,0.329,111.0,117.2,-5.89
1243,Houston Rockets,HOU,2023,22,60,0.268,110.7,118.6,-7.62
1244,San Antonio Spurs,SAS,2023,22,60,0.268,113.0,123.1,-9.82


In [4]:
target = pd.read_csv('data/Standings_1979-2023.csv')[['Tm', 'Year', 'W/L%']]
target.to_csv('data/target_1979-2023.csv', index = None)

## Create team strength features

In [6]:
data = pd.read_csv('data/Standings_1979-2023.csv')
data.head()

,Team,Tm,Year,W,L,W/L%,PS/G,PA/G,SRS
0,Washington Bullets,WSB,1979,54,28,0.659,114.9,109.9,4.75
1,Seattle SuperSonics,SEA,1979,52,30,0.634,106.6,103.9,2.69
2,Phoenix Suns,PHO,1979,50,32,0.610,115.4,111.7,3.55
3,San Antonio Spurs,SAS,1979,48,34,0.585,119.3,114.1,4.97
4,Kansas City Kings,KCK,1979,48,34,0.585,113.1,110.2,2.73
